In [1]:
# Add parent directory to working directory. 
import sys 
sys.path.append('..')

In [2]:
import math
import quaternion
import numpy as np 
from astropy import units 

In [3]:
from measures.api import Length, Angle, Speed

# Vector3 
Basically a numpy array.  It is restricted to only being 3 elements long and must be of numeric type (float precision is stored).

In [4]:
from kinematics.utils import Vector3, UnitVector3

In [5]:
Vector3(10, 10, '10')

Vector3(10.0, 10.0, 10.0)

In [6]:
Vector3([10, 2, 3])

Vector3(10.0, 2.0, 3.0)

In [7]:
# Can also get unit vectors (magnitude = 1)
UnitVector3(10, 34, 3)

UnitVector3(0.28116077855776667, 0.9559466470964066, 0.08434823356733)

# Coordinate Frames and Points

## Base frame and cartesian frames 

To start, you must initialize a base frame.  This is a frame that represents where (0,0,0) is and what the global x,y,z axes are.  

Then, instance CartesianFrames in refrence to the base frame.  These will be the workable coordinate frames--the ones you describe points with.   

Points are described in reference to a CartesianFrame 

In [8]:
from kinematics.utils import BaseFrame, CartesianFrame, Point 

In [9]:
world = BaseFrame()

In [10]:
my_frame = CartesianFrame(base_frame=world)
my_frame.origin.coords

Vector3(0.0, 0.0, 0.0)

In [11]:
p1 = Point([1, 2, 3], frame=my_frame, dimension_unit=units.imperial.ft)
p1


Point[my_frame(Cartesian)](1.000000000 ft, 2.000000000 ft, 3.000000000 ft)

In [12]:
# Can translate a point by calling .translate().  Returns a new point object.
p1.translate([1,1,1])

Point[my_frame(Cartesian)](2.000000000 ft, 3.000000000 ft, 4.000000000 ft)

In [13]:
# Mutate a point inplace by calling x, y, z attrs (does NOT return new point)
print(p1)
p1.x = 4
p1.y += 100
p1.z *= 10
print(p1)

Point[my_frame(Cartesian)](1.000 ft, 2.000 ft, 3.000 ft)
Point[my_frame(Cartesian)](4.000 ft, 102.000 ft, 30.000 ft)


## Fancy coord frames

You can translate and rotate cartesian frames with respect to their base frame.  Points that are defined with respect to one frame can be represented with respect to other frames.  

There are a few presets available with the `axes_convention` input arg.  

In [14]:
rotation = [Angle(i*units.deg) for i in [45, 180, 90]]

best_frame_ever = CartesianFrame(base_frame=world, 
                              translation=(10, 10, 10), 
                              orientation=rotation,
                              dimension_unit=units.m,
                              name='best')
best_frame_ever

CartesianFrame(best, origin: Point[world(Base)](10.000 m, 10.000 m, 10.000 m), orientation: [0.78539816 3.14159265 1.57079633])

In [15]:
ned = CartesianFrame(axes_convention='ned')
enu = CartesianFrame(axes_convention='enu')
# ENU is considered equivalent to the base_frame.

In [16]:
# Note.  this operation returns a new point.  p1 is not mutated
best_point_ever = p1.to_frame(best_frame_ever)
best_point_ever

Point[best(Cartesian)](79.296464556 ft, 84.953318806 ft, -20.000000000 ft)

In [17]:
p1.to_frame(ned)

Point[ned(Cartesian)](102.000000000 ft, 4.000000000 ft, -30.000000000 ft)

# Velocity vector 

The Velocity object represents a velocity vector.  It has no position in 3D space, only a magnitude and direction.  

You can obtain the azimuth and elevation angles from a Velocity object with `.get_azimuth_elevation()`.  

In [18]:
from kinematics.utils import Velocity 

In [19]:
v = Velocity(vel=[10, 1, 3], 
         frame=best_frame_ever, 
         dimension_unit=units.km/units.s)

print(v)  # Shorter.  Rounds to .3f 
v

Velocity[best(Cartesian)](10.000 km / s, 1.000 km / s, 3.000 km / s)


Velocity[best(Cartesian)](10.000000000 km / s, 1.000000000 km / s, 3.000000000 km / s)

In [20]:
print(v.to_frame(ned))

Velocity[ned(Cartesian)](7.778 km / s, -6.364 km / s, 3.000 km / s)


In [21]:
v.vector
# p1.coords

Vector3(10.0, 1.0, 3.0)

In [22]:
v.get_azimuth_elevation()

(<Angle 5.71059314 deg>, <Angle 16.62095127 deg>)

# State 

State packages a position and velocity vector together.  As of now, the position and velocity must be described by the same frame to make a state of out them.  This could change to be more dynamic if there is a need for it.  
  

TODO:  add orientation to the state object. 

In [23]:
from kinematics.utils import State

In [24]:
State(best_point_ever, v)

State(Point[best(Cartesian)](79.296 ft, 84.953 ft, -20.000 ft), Velocity[best(Cartesian)](10.000 km / s, 1.000 km / s, 3.000 km / s))